# Analysis Notebook

## Package imports and configurations

In [1]:
# imports
import os

from pathlib import Path

import pandas as pd
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt

from rich import print
from tqdm import tqdm

In [2]:
# matplotlib configuration
cstyle_url = r"https://gist.githubusercontent.com/CalvinSprouse/30bba681da1dd0023e948ca207fad0fc/raw/80a1ae94031873a57c9624103e83f5caf175ac86/cstyle.mplstyle"
plt.style.use(["default", cstyle_url])

# configure figure directory
figure_dir = Path(".figures")
if not figure_dir.exists(): figure_dir.mkdir()

## Data loading and initial processing

In [3]:
# define the location of data files
mat_file = Path("file.mat")

# load data into dataframes / arrays
mat_data = sp.io.loadmat(mat_file)

## Main Analysis